In [6]:
# Install PyTorch with CUDA support (for GPU acceleration)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install transformers, bitsandbytes (for 4-bit quantization), and accelerate
!pip install transformers bitsandbytes accelerate


Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configure 4-bit quantization to reduce memory usage
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

# Load the CodeLlama model and tokenizer
model_name = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically uses GPU if available
    quantization_config=quantization_config,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def analyze_code(code):
    """Analyze student code for errors, inefficiencies, or misconceptions."""
    prompt = f"""
    You are a Python teaching assistant.
    Analyze the following code for errors, inefficiencies, or misconceptions.
    Explain the issues and suggest a question to assess the student's understanding.
    Code:
    ```python
    {code}
    ```
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.3)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_prompts(code):
    """Generate Socratic questions to assess the student's understanding."""
    prompt = f"""
    Generate 3 Socratic questions to assess a student's understanding of this code:
    ```python
    {code}
    ```
    Focus on conceptual gaps, not syntax errors.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.5)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# Example student code with intentional errors
student_code = """
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

a = 5
print(factorial(a))
print(a / b)  # Error: 'b' is not defined
"""

# Run analysis
analysis = analyze_code(student_code)
print("=== CODE ANALYSIS ===\n", analysis)

# Generate prompts
prompts = generate_prompts(student_code)
print("\n=== SUGGESTED PROMPTS ===\n", prompts)


In [ ]:
with open("codellama_output.txt", "w") as f:
    f.write("=== CODE ANALYSIS ===\n" + analysis + "\n\n")
    f.write("=== SUGGESTED PROMPTS ===\n" + prompts + "\n")
